In [1]:
!pip install imbalanced-learn


In [14]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler


In [3]:
url = "https://raw.githubusercontent.com/AnjulaMehto/Sampling_Assignment/main/Creditcard_data.csv"
df = pd.read_csv(url)

df.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [16]:
X = df.drop('Class', axis=1)
y = df['Class']

print("Class Distribution:")
print(y.value_counts())


Class Distribution:
Class
0    763
1      9
Name: count, dtype: int64


In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)


In [18]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [19]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Support Vector Machine": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier()
}


In [24]:
def systematic_sampling(X, y, step=10):
    idx = np.arange(0, len(X), step)
    return X.iloc[idx], y.iloc[idx]


In [23]:
def random_sampling(X, y):
    return train_test_split(X, y, test_size=0.3, random_state=42)


In [25]:
def stratified_sampling(X, y):
    return train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)


In [26]:
def cluster_sampling(X, y):
    temp = X.copy()
    temp['Class'] = y
    sampled = temp.groupby('Class').sample(frac=1.0, random_state=42)
    return sampled.drop('Class', axis=1), sampled['Class']


In [27]:
def bootstrap_sampling(X, y):
    X_res, y_res = resample(X, y, replace=True, random_state=42)
    return X_res, y_res


In [28]:
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_train_scaled, y_train)


In [29]:
results = pd.DataFrame(index=models.keys(), columns=[
    "Original Data",
    "Random Sampling",
    "Systematic Sampling",
    "Stratified Sampling",
    "Cluster Sampling",
    "Bootstrap Sampling",
    "SMOTE Oversampling"
])


In [31]:
for model_name, model in models.items():

    # Original Data
    model.fit(X_train_scaled, y_train)
    results.loc[model_name, "Original Data"] = round(
        accuracy_score(y_test, model.predict(X_test_scaled)), 4
    )

    # Random Sampling
    Xr_tr, Xr_te, yr_tr, yr_te = random_sampling(X, y)
    Xr_tr = scaler.fit_transform(Xr_tr)
    Xr_te = scaler.transform(Xr_te)
    model.fit(Xr_tr, yr_tr)
    results.loc[model_name, "Random Sampling"] = round(
        accuracy_score(yr_te, model.predict(Xr_te)), 4
    )

    # Stratified Sampling
    Xs_tr, Xs_te, ys_tr, ys_te = stratified_sampling(X, y)
    Xs_tr = scaler.fit_transform(Xs_tr)
    Xs_te = scaler.transform(Xs_te)
    model.fit(Xs_tr, ys_tr)
    results.loc[model_name, "Stratified Sampling"] = round(
        accuracy_score(ys_te, model.predict(Xs_te)), 4
    )

    # Bootstrap Sampling
    Xb, yb = resample(X_train_scaled, y_train, replace=True, random_state=42)
    model.fit(Xb, yb)
    results.loc[model_name, "Bootstrap Sampling"] = round(
        accuracy_score(y_test, model.predict(X_test_scaled)), 4
    )

    # SMOTE Oversampling
    model.fit(X_smote, y_smote)
    results.loc[model_name, "SMOTE Oversampling"] = round(
        accuracy_score(y_test, model.predict(X_test_scaled)), 4
    )


In [32]:
print("="*100)
print("ACCURACY SCORES - MODELS vs SAMPLING TECHNIQUES")
print("="*100)
results


ACCURACY SCORES - MODELS vs SAMPLING TECHNIQUES


,Original Data,Random Sampling,Systematic Sampling,Stratified Sampling,Cluster Sampling,Bootstrap Sampling,SMOTE Oversampling
Logistic Regression,0.9871,0.9828,NaN,0.9871,NaN,0.9871,0.9397
Random Forest,0.9871,0.9871,NaN,0.9871,NaN,0.9871,0.9914
Support Vector Machine,0.9871,0.9871,NaN,0.9871,NaN,0.9871,0.9655
K-Nearest Neighbors,0.9871,0.9871,NaN,0.9871,NaN,0.9871,0.9483
Decision Tree,0.9784,0.9871,NaN,0.9741,NaN,0.9871,0.9828
